In [ ]:
import pandas as pd
import pickle
from flask import Flask, request, jsonify
from surprise import SVD

# Initialisation de l'application Flask
app = Flask(__name__)

# Charger le DataFrame df_merged depuis le fichier compressé
df_merged = pd.read_pickle("df_merged_compressed.pkl", compression="gzip")

# Préparation des données d'interaction
interaction = df_merged[['user_id', 'article_id', 'session_id']].groupby(by=['user_id', 'article_id'], as_index=False).agg('count')
interaction.rename(columns={'session_id': 'rating'}, inplace=True)
interaction['rating'] = (interaction['rating'] - interaction['rating'].min()) / (interaction['rating'].max() - interaction['rating'].min())

# Charger le modèle SVD sauvegardé
with open('best_svd_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Fonction de recommandation
def get_recommendations(user_id, model, interaction_df, n=5):
    articles = set(interaction_df['article_id'].unique())
    predictions = [model.predict(user_id, article).est for article in articles]
    top_articles = sorted(zip(articles, predictions), key=lambda x: x[1], reverse=True)[:n]
    return [article for article, _ in top_articles]

# Route pour obtenir des recommandations
@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = request.args.get('user_id', default=1, type=int)
    n = request.args.get('n', default=5, type=int)
    recommended_articles = get_recommendations(user_id, loaded_model, interaction, n)
    return jsonify({'user_id': user_id, 'recommendations': recommended_articles})

if __name__ == "__main__":
    app.run(debug=True)

